In [368]:
import random
import string
import numpy as np
import pandas as pd
from tabulate import tabulate

### Config

In [369]:
DISH_COUNT = 300
MIN_CALORIES_PER_DISH = 100
MAX_CALORIES_PER_DISH = 1200
CALORIES_PER_DAY = 2000
DAYS_PER_MENU = 7
DISHES_PER_DAY = 3

PENALTY = 400
POPULATION_SIZE = 20
MATING_POPULATION_SIZE = 10
ITERATIONS = 3000
PROBABILITY_OF_RANDOM_MUTATE = 0.1
PROBABILITY_OF_SWAP_MUTATE = 0.1

REAL_DATA = True

#### Helper functions

In [370]:
def visualize_menu(menu):
    print("Score: " + str(score_menu(menu)))
    headers = ["Day", "Calories", "Dishes"]
    calories = count_calories_per_day(menu)
    table = []
    for day in range(DAYS_PER_MENU):
        table.append([day, calories[day], menu[day]])
    print(tabulate(table, headers, tablefmt="fancy_grid"))

#### Test data

In [371]:
def fake_data():
    dishes = {}
    for _ in range(DISH_COUNT):
        name = ''.join(random.choices(string.ascii_lowercase, k=6))
        dishes[name] = random.randint(MIN_CALORIES_PER_DISH, MAX_CALORIES_PER_DISH)
    dishes = list(dishes.items())
    return dishes

#### Real data

In [372]:
def real_data():
    dishes = {}
    data = pd.read_csv('data.csv', delimiter=',', encoding='utf-8', low_memory=False)
    df = pd.DataFrame(data, columns=['name', 'Food Group', "calories_portion"])

    while len(dishes) != DISH_COUNT:
        random_choice = df.sample()
        if MIN_CALORIES_PER_DISH < random_choice.calories_portion.values[0] < MAX_CALORIES_PER_DISH:
            dishes[random_choice.name.values[0]] = random_choice.calories_portion.values[0]
    dishes = list(dishes.items())
    return dishes

In [373]:
dishes = {}
if REAL_DATA:
    dishes = real_data()
else:
    dishes = fake_data()
print(*dishes[:7], sep='\n')

('Popeyes Fried Chicken Mild Breast Meat And Skin With Breading', 531.56)
('Prunes (Dried Plums)', 417.6)
('Fish Salmon Chum Canned Drained Solids With Bone', 119.85)
('Knish Meat', 174.5)
('Tilapia Coated Baked Or Broiled Made With Margarine', 212.44)
('Roll Sweet Frosted', 216.0)
('Chicken Or Turkey And Vegetables Including Carrots Broccoli And/or Dark-Green Leafy; No Potatoes Cheese Sauce', 296.31)


#### Sample random menu

In [374]:
def create_random_menu():
    menu = np.ndarray((DAYS_PER_MENU, DISHES_PER_DAY), tuple)
    for day in range(DAYS_PER_MENU):
        for dish_index in range(DISHES_PER_DAY):
            menu[day, dish_index] = random.choice(dishes)
    return menu

In [375]:
def count_calories_per_day(menu):
    calories = []
    for day in range(DAYS_PER_MENU):
        calories.append(0)
        for dish_index in range(DISHES_PER_DAY):
            calories[-1] += menu[day, dish_index][1]

    return calories


def count_attr_per_day(menu):
    names = np.reshape(menu, 21)
    unique, counts = np.unique(names, return_counts=True)
    names_dict = dict(zip(unique, counts))

    attr = []
    for day in range(DAYS_PER_MENU):
        attr.append(0)
        for dish_index in range(DISHES_PER_DAY):
            attr[-1] += (names_dict[menu[day, dish_index]] - 1) * PENALTY

    return attr


def score_menu(menu):
    differences = [calories - CALORIES_PER_DAY for calories in count_calories_per_day(menu)]
    penalty = [attr ** 2 for attr in count_attr_per_day(menu)]
    squares = [calories ** 2 for calories in differences]

    return round(sum(squares + penalty))


def check_duplications(menu):
    names = np.reshape(menu, 21)
    unique, counts = np.unique(names, return_counts=True)
    is_repeating = dict(zip(unique, counts))

    for elem in is_repeating:
        if is_repeating[elem] > 2:
            print("duplicants detected")
            return
    print("No duplicants")

In [376]:
sample_menu = create_random_menu()
visualize_menu(sample_menu)

Score: 10880340
╒═══════╤════════════╤════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╕
│   Day │   Calories │ Dishes                                                                                                                     │
╞═══════╪════════════╪════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╡
│     0 │    1369.67 │ [('Chicken Pot Pie Frozen Entree Prepared', 616.08)                                                                        │
│       │            │  ('Restaurant Mexican Cheese Tamales', 652.32)                                                                             │
│       │            │  ('Grapefruit Juice White Frozen Concentrate Unsweetened Diluted With 3 Volume Water', 101.27)]                            │
├───────┼────────────┼──────────────────────────────────────────────────────────────────────────

### Algorithm

In [377]:
def crossover(first_menu, second_menu):
    child = first_menu.copy()
    for day_index in range(DAYS_PER_MENU):
        if random.choice((0, 1)) == 1:
            child[day_index] = second_menu[day_index]
    return child


def random_mutate(menu):
    menu[random.randint(0, DAYS_PER_MENU - 1), random.randint(0, DISHES_PER_DAY - 1)] = random.choice(dishes)
    return menu


def swap_mutate(menu):
    first_day_index = random.randint(0, DAYS_PER_MENU - 1)
    second_day_index = random.randint(0, DAYS_PER_MENU - 1)
    dish_index = random.randint(0, DISHES_PER_DAY - 1)
    menu[first_day_index, dish_index], menu[second_day_index, dish_index] = menu[second_day_index, dish_index], menu[
        first_day_index, dish_index]
    return menu

In [378]:
def initial_population():
    population = []
    for _ in range(POPULATION_SIZE):
        population.append(create_random_menu())
    return population

In [379]:
def genetic_algorithm():
    population = initial_population()
    for i in range(ITERATIONS):
        population.sort(key=score_menu)
        mating_population = population[:MATING_POPULATION_SIZE]

        offspring_population = []
        for _ in range(POPULATION_SIZE - MATING_POPULATION_SIZE):
            offspring_population.append(crossover(random.choice(mating_population), random.choice(mating_population)))
            if random.random() < PROBABILITY_OF_RANDOM_MUTATE:
                offspring_population[-1] = random_mutate(offspring_population[-1])
            if random.random() < PROBABILITY_OF_SWAP_MUTATE:
                offspring_population[-1] = swap_mutate(offspring_population[-1])

        population = mating_population + offspring_population

    population.sort(key=score_menu)
    best_individual, worst_individual = population[0], population[-1]

    return best_individual, worst_individual

### Example

In [380]:
best_individual, worst_individual = genetic_algorithm()
visualize_menu(best_individual)
check_duplications(best_individual)

Score: 451284
╒═══════╤════════════╤═════════════════════════════════════════════════════════════════════════════════════════════════════╕
│   Day │   Calories │ Dishes                                                                                              │
╞═══════╪════════════╪═════════════════════════════════════════════════════════════════════════════════════════════════════╡
│     0 │    1752    │ [('Burger King Croissanwich With Sausage Egg And Cheese', 526.68)                                   │
│       │            │  ('Chicken Fried With Potatoes Vegetable Dessert Frozen Meal', 546.0)                               │
│       │            │  ('Pastry Made With Bean Paste And Salted Egg Yolk Filling Baked', 679.32)]                         │
├───────┼────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────┤
│     1 │    1728.93 │ [('McDonalds Big Mac', 562.83)                                                          

### Experiments

In [381]:
def visualize_experiment(parameters, measurements):
    headers = [parameters, "Best", "Worst"]
    table = []
    for measurement in measurements:
        table.append(measurement)
    print(tabulate(table, headers, tablefmt="fancy_grid"))

In [382]:
def experiment_population_size():
    results = []
    for population_size in [10, 20, 50, 100, 150, 250, 500, 1000, 2000]:
        POPULATION_SIZE = population_size
        MATING_POPULATION_SIZE = population_size // 2
        best_individual, worst_individual = genetic_algorithm()
        results.append([population_size, score_menu(best_individual), score_menu(worst_individual)])
    return results


visualize_experiment("Population size", experiment_population_size())

╒═══════════════════╤════════╤═════════╕
│   Population size │   Best │   Worst │
╞═══════════════════╪════════╪═════════╡
│                10 │ 409706 │  560485 │
├───────────────────┼────────┼─────────┤
│                20 │ 433988 │  814278 │
├───────────────────┼────────┼─────────┤
│                50 │ 413144 │  675037 │
├───────────────────┼────────┼─────────┤
│               100 │ 420020 │  953587 │
├───────────────────┼────────┼─────────┤
│               150 │ 421774 │  421774 │
├───────────────────┼────────┼─────────┤
│               250 │ 386786 │  608673 │
├───────────────────┼────────┼─────────┤
│               500 │ 396441 │  616232 │
├───────────────────┼────────┼─────────┤
│              1000 │ 417937 │  417937 │
├───────────────────┼────────┼─────────┤
│              2000 │ 386546 │  746069 │
╘═══════════════════╧════════╧═════════╛


In [383]:
def experiment_iterations():
    results = []
    for iterations in [10, 20, 50, 100, 150, 250, 500, 1000, 2000]:
        ITERATIONS = iterations
        best_individual, worst_individual = genetic_algorithm()
        results.append([iterations, score_menu(best_individual), score_menu(worst_individual)])
    return results


visualize_experiment("Iterations", experiment_iterations())

╒══════════════╤════════╤═════════╕
│   Iterations │   Best │   Worst │
╞══════════════╪════════╪═════════╡
│           10 │ 424240 │  424240 │
├──────────────┼────────┼─────────┤
│           20 │ 433307 │  443635 │
├──────────────┼────────┼─────────┤
│           50 │ 418167 │  443545 │
├──────────────┼────────┼─────────┤
│          100 │ 388732 │  776799 │
├──────────────┼────────┼─────────┤
│          150 │ 426347 │  802763 │
├──────────────┼────────┼─────────┤
│          250 │ 388030 │  423260 │
├──────────────┼────────┼─────────┤
│          500 │ 456000 │  828761 │
├──────────────┼────────┼─────────┤
│         1000 │ 418165 │ 1038130 │
├──────────────┼────────┼─────────┤
│         2000 │ 394147 │  394147 │
╘══════════════╧════════╧═════════╛
